### rental-prices-selangor-kl data wrangling

### Imports

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 

pd.options.display.float_format = '{:.0f}'.format

ERROR! Session/line number was not unique in database. History logging moved to new session 511


### Goal

The goal is the analyse what factors determine the rental prices in Selangor and KL, as well as creating a model to predict the rental prices based on the property factors

### Data Source

The data is sourced from Kaggle where it was scraped from mudah.my

### Data Loading

In [2]:
path = "../data/mudah-apartment-kl-selangor.csv"

In [3]:
data = pd.read_csv(path)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19991 entries, 0 to 19990
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ads_id                 19991 non-null  int64  
 1   prop_name              19043 non-null  object 
 2   completion_year        10806 non-null  float64
 3   monthly_rent           19989 non-null  object 
 4   location               19991 non-null  object 
 5   property_type          19991 non-null  object 
 6   rooms                  19985 non-null  object 
 7   parking                14289 non-null  float64
 8   bathroom               19985 non-null  float64
 9   size                   19991 non-null  object 
 10  furnished              19986 non-null  object 
 11  facilities             17782 non-null  object 
 12  additional_facilities  14043 non-null  object 
 13  region                 19991 non-null  object 
dtypes: float64(3), int64(1), object(10)
memory usage: 2.1+

Assuming that each property should have a unique ads_id, we remove any duplicate listings

In [5]:
data = data.drop_duplicates(['ads_id'], keep='first')

Data shape is:

In [6]:
data.shape

(19978, 14)

### Data Type Conversion

Ensure that the data types of each column is correct before we perform data exploration since some data are numeric but formatted in string. This would cause difficulties in checking for outliers.

In [7]:
data.dtypes

ads_id                     int64
prop_name                 object
completion_year          float64
monthly_rent              object
location                  object
property_type             object
rooms                     object
parking                  float64
bathroom                 float64
size                      object
furnished                 object
facilities                object
additional_facilities     object
region                    object
dtype: object

Let's take a look at the dataset

In [8]:
data.head()

,ads_id,prop_name,completion_year,monthly_rent,location,property_type,rooms,parking,bathroom,size,furnished,facilities,additional_facilities,region
0,100323185,The Hipster @ Taman Desa,2022,RM 4 200 per month,Kuala Lumpur - Taman Desa,Condominium,5,2,6,1842 sq.ft.,Fully Furnished,"Minimart, Gymnasium, Security, Playground, Swi...","Air-Cond, Cooking Allowed, Washing Machine",Kuala Lumpur
1,100203973,Segar Courts,NaN,RM 2 300 per month,Kuala Lumpur - Cheras,Condominium,3,1,2,1170 sq.ft.,Partially Furnished,"Playground, Parking, Barbeque area, Security, ...","Air-Cond, Cooking Allowed, Near KTM/LRT",Kuala Lumpur
2,100323128,Pangsapuri Teratak Muhibbah 2,NaN,RM 1 000 per month,Kuala Lumpur - Taman Desa,Apartment,3,NaN,2,650 sq.ft.,Fully Furnished,"Minimart, Jogging Track, Lift, Swimming Pool",NaN,Kuala Lumpur
3,100191767,Sentul Point Suite Apartment,2020,RM 1 700 per month,Kuala Lumpur - Sentul,Apartment,2,1,2,743 sq.ft.,Partially Furnished,"Parking, Playground, Swimming Pool, Squash Cou...","Cooking Allowed, Near KTM/LRT, Washing Machine",Kuala Lumpur
4,97022692,Arte Mont Kiara,NaN,RM 1 299 per month,Kuala Lumpur - Mont Kiara,Service Residence,1,1,1,494 sq.ft.,Not Furnished,"Parking, Security, Lift, Swimming Pool, Playgr...",Air-Cond,Kuala Lumpur


`monthly_rent` is a numerical value but formatted with "RM" and "per month". 

In [9]:
data['monthly_rent'] = data['monthly_rent'].str.replace(" ", '').str.extract('(\d+)')
data['monthly_rent'] = pd.to_numeric(data['monthly_rent'])

The same thing is done for `size`

In [10]:
data['size'] = data['size'].str.replace(" ", '').str.extract('(\d+)')
data['size'] =  pd.to_numeric(data['size'])

In [11]:
data.head()

,ads_id,prop_name,completion_year,monthly_rent,location,property_type,rooms,parking,bathroom,size,furnished,facilities,additional_facilities,region
0,100323185,The Hipster @ Taman Desa,2022,4200,Kuala Lumpur - Taman Desa,Condominium,5,2,6,1842,Fully Furnished,"Minimart, Gymnasium, Security, Playground, Swi...","Air-Cond, Cooking Allowed, Washing Machine",Kuala Lumpur
1,100203973,Segar Courts,NaN,2300,Kuala Lumpur - Cheras,Condominium,3,1,2,1170,Partially Furnished,"Playground, Parking, Barbeque area, Security, ...","Air-Cond, Cooking Allowed, Near KTM/LRT",Kuala Lumpur
2,100323128,Pangsapuri Teratak Muhibbah 2,NaN,1000,Kuala Lumpur - Taman Desa,Apartment,3,NaN,2,650,Fully Furnished,"Minimart, Jogging Track, Lift, Swimming Pool",NaN,Kuala Lumpur
3,100191767,Sentul Point Suite Apartment,2020,1700,Kuala Lumpur - Sentul,Apartment,2,1,2,743,Partially Furnished,"Parking, Playground, Swimming Pool, Squash Cou...","Cooking Allowed, Near KTM/LRT, Washing Machine",Kuala Lumpur
4,97022692,Arte Mont Kiara,NaN,1299,Kuala Lumpur - Mont Kiara,Service Residence,1,1,1,494,Not Furnished,"Parking, Security, Lift, Swimming Pool, Playgr...",Air-Cond,Kuala Lumpur


### Data Exploration

Check for any missing values that we have to deal with

In [12]:
missing = data.isna().sum(axis=0)
missing.name = 'Missing Count'
missing = missing.to_frame()
missing = missing[missing['Missing Count']>0]
missing.sort_values("Missing Count", ascending=False)

,Missing Count
completion_year,9180
additional_facilities,5944
parking,5698
facilities,2208
prop_name,948
rooms,6
bathroom,6
furnished,5
monthly_rent,2


In [13]:
data.describe()

,ads_id,completion_year,monthly_rent,parking,bathroom,size
count,19978,10798,19976,14280,19972,19978
mean,99706807,2015,2270,1,2,5925
std,3483246,7,22420,1,1,707490
min,16525511,1977,70,1,1,1
25%,99840140,2012,1100,1,2,750
50%,100212675,2017,1400,1,2,886
75%,100617730,2020,1800,2,2,1044
max,100854617,2025,2400000,10,8,99999999


We can see that the maximum amount for `monthly_rent` is 2400000 whereas the minimum amount is 70. This could be deemed an outlier as it is significantly higher than the mean for rent. The maximum number for `parking` is 10 which could also be deemed as an outlier/error. The maximum number for `bathroom` is 8, which will be considered an outlier. The maximum size for `size` is 99999999, which is significantly higher than the mean and thus will also be considered an outlier.

Let's look at which rows contain the outliers. Starting with `monthly_rent`:

Since mudah.my is a platform that users may use to sell properties, there is a high possibility that certain listings scraped aren't intended for rent. As we can observe, there are properties with `monthly_rent` values over 20,000. That is unrealistic and we will remove them, assuming they are properties for sale.

In [22]:
data[data['monthly_rent']>=20000].sort_values(by='monthly_rent').head()

,ads_id,prop_name,completion_year,monthly_rent,location,property_type,rooms,parking,bathroom,size,furnished,facilities,additional_facilities,region
15754,100676946,Taman Bukit Mewah (Kajang),NaN,45000,Selangor - Kajang,Flat,2.0,NaN,1,603,Not Furnished,NaN,"Cooking Allowed, Near KTM/LRT",Selangor
18629,100553520,Pangsapuri Kiambang (Taman Bukit Subang),NaN,95000,Selangor - Shah Alam,Apartment,3.0,NaN,2,750,Partially Furnished,"Playground, Security, Parking, Multipurpose ha...",Cooking Allowed,Selangor
14207,99737409,Mutiara (Beranang),NaN,105000,Selangor - Beranang,Flat,3.0,1,2,700,Not Furnished,"Parking, Security, Lift, Playground, Sauna, Mi...",Cooking Allowed,Selangor
10885,99866938,Harmoni Apartment,NaN,125000,Selangor - Damansara Damai,Apartment,3.0,NaN,2,650,Partially Furnished,Parking,NaN,Selangor
12113,100802221,Rumah Pangsa Impian (Saujana Putra),NaN,127898,Selangor - Kuala Langat,Flat,3.0,NaN,2,650,Not Furnished,"Security, Playground, Minimart","Cooking Allowed, Near KTM/LRT",Selangor


In [28]:
data = data[(data['monthly_rent'] < 25000) & (data['monthly_rent'] > 70)]

In [62]:
data = data[(data['size']< 30000) & (data['size'] > 50)]

### Handling Missing Values

In [45]:
data.isna().sum()

ads_id                      0
prop_name                 941
completion_year          9146
monthly_rent                0
location                    0
property_type               0
rooms                       5
parking                  5683
bathroom                    5
size                        0
furnished                   3
facilities               2204
additional_facilities    5913
region                      0
dtype: int64

`parking` has null values, but when we look at the unique values of the column, we can see that there are no 0's. We can assume that properties with null values for `parking` have no parking spaces.

In [46]:
data['parking'].unique()

array([ 2.,  1., nan,  3.,  5.,  4.,  9.,  6., 10.,  7.])

In [47]:
data['parking'].fillna(0, inplace=True)

The same can be assumed for null values in `rooms` and `bathrooms`.

In [48]:
data['rooms'].unique()

array(['5', '3', '2', '1', '4', '7', '6', 'More than 10', nan, '9', '3.0',
       '4.0', '2.0', '1.0', '5.0', '9.0', '6.0', '7.0', '10.0'],
      dtype=object)

In [49]:
data['rooms'].fillna(0, inplace=True)
data['bathroom'].fillna(0, inplace=True)

We can also assume that the null values in `facilities` and `additional_facilities` are due to there being no facilities at all. Instead of keeping them as null values, we replace them with 'None'. This is so that these rows can be used for analysis and in the prediction model as well.

In [50]:
data['facilities'].fillna("None", inplace=True)
data['additional_facilities'].fillna("None", inplace=True)

In [51]:
data.isna().sum()

ads_id                      0
prop_name                 941
completion_year          9146
monthly_rent                0
location                    0
property_type               0
rooms                       0
parking                     0
bathroom                    0
size                        0
furnished                   3
facilities                  0
additional_facilities       0
region                      0
dtype: int64

I decided to simply drop the rows where `furnished` was left null as there were only 3 rows.

In [52]:
data.dropna(subset=['monthly_rent', 'furnished'], inplace=True)

### Data Manipulation

The values for `facilities` and `additional_facilities` are strings, listing each facility available in the rented property. However, this isn't very usable for data analysis and prediction. Hence why we decided to create dummy variables out of the facilities and set their values as Boolean to represent whether the property offers the facilities. 

First we get a list of all the facilities offered by the properties.

In [53]:
data.shape

(19916, 14)

In [54]:
data['facilities'].size

19916

In [55]:
all_facilities = []
for i in range(data.shape[0]):
    all_facilities.extend(data['facilities'].iloc[i].split(", "))
    all_facilities.extend(data['additional_facilities'].iloc[i].split(", "))

To remove the duplicate facilities in the list, we convert it into a set.

In [56]:
facilities_set = set(all_facilities)

However, there are some random facilities that do not make sense, such as '', '10', '11' and '6'. This could be due to inaccurate web scraping. We will just remove them from the set of facilities.

In [57]:
facilities_set

{'',
 '10',
 '11',
 '6',
 'Air-Cond',
 'Barbeque area',
 'Club house',
 'Cooking Allowed',
 'Gymnasium',
 'Internet',
 'Jogging Track',
 'Lift',
 'Minimart',
 'Multipurpose hall',
 'Near KTM/LRT',
 'None',
 'Parking',
 'Playground',
 'Sauna',
 'Security',
 'Squash Court',
 'Swimming Pool',
 'Tennis Court',
 'Washing Machine'}

In [58]:
facilities_set.remove("")
facilities_set.remove("6")
facilities_set.remove("10")
facilities_set.remove("11")

We then create the columns to hold the boolean value for each facility based on whether or not the property includes them

In [59]:
data = data.assign(AC = lambda x: data['additional_facilities'].str.contains('Air-Cond'),
                   Cooking = lambda x: data['additional_facilities'].str.contains('Cooking Allowed'),
                   Internet = lambda x: data['additional_facilities'].str.contains('Internet'),
                   Pub_Trsp = lambda x: data['additional_facilities'].str.contains('Near KTM/LRT'),
                   No_Afac = lambda x: data['additional_facilities'].str.contains('None'),
                   Washing_Machine = lambda x: data['additional_facilities'].str.contains('Washing Machine'),
                   BBQ = lambda x: data['facilities'].str.contains('Barbeque area'),
                   Club_House = lambda x: data['facilities'].str.contains('Club house'),
                   Gym = lambda x: data['facilities'].str.contains('Gymnasium'),
                   Jog = lambda x: data['facilities'].str.contains('Jogging Track'),
                   Lift = lambda x: data['facilities'].str.contains('Lift'),
                   Minimart = lambda x: data['facilities'].str.contains('Minimart'),
                   MP_Hall = lambda x: data['facilities'].str.contains('Multipurpose hall'),
                   No_Fac = lambda x: data['facilities'].str.contains('None'),
                   Parking = lambda x: data['facilities'].str.contains('Parking'),
                   Playground = lambda x: data['facilities'].str.contains('Playground'),
                   Sauna = lambda x: data['facilities'].str.contains('Sauna'),
                   Security = lambda x: data['facilities'].str.contains('Security'),
                   Squash = lambda x: data['facilities'].str.contains('Squash Court'),
                   Pool = lambda x: data['facilities'].str.contains('Swimming Pool'),
                   Tennis = lambda x: data['facilities'].str.contains('Tennis Court'),)
                   

The final shape of the data is:

In [60]:
data.shape

(19916, 35)

In [63]:
data.to_csv("../data/Rental_Price_cleaned.csv")